In [2]:
from bhpwave.spline import CubicSpline, BicubicSpline, TricubicSpline
import numpy as np

In [4]:
import platform

In [13]:
x = np.linspace(0.5, 1, 257)
y = np.linspace(-1, 1, 257)
z = np.linspace(2, 3, 513)
XYZ = np.meshgrid(x, y, z, indexing='ij')

In [14]:
def test_func(x, y, z):
    return np.sin(x)*np.cos(y)/(1-z**2)

In [15]:
fvec = test_func(*XYZ)

In [16]:
(2**31-1)/64/257/257

508.02331578638587

In [17]:
trispl = TricubicSpline(x, y, z, fvec)

RuntimeError: vector

In [35]:
1-trispl(0.51, 0.4, 2.4)/test_func(0.51, 0.4, 2.4)

9.819367541297197e-12

# Scratch

In [6]:
# test = []
# for i in range(x.shape[0]):
#     YZ = np.meshgrid(y, z, indexing='ij')
#     ftemp = test_func(x[i], *YZ)
#     test.append(BicubicSpline(y, z, ftemp))

In [7]:
coefftest = np.empty((x.shape[0], y.shape[0], z.shape[0], 4, 4))
for i in range(len(test)):
    for j in range(y.shape[0]):
        for k in range(z.shape[0]):
            for iy in range(4):
                for iz in range(4):
                    coefftest[i, j, k, iy, iz] = test[i].coeff(j, k, iy, iz)


In [8]:
coefftest2 = np.empty((x.shape[0], y.shape[0], z.shape[0], 4, 4, 4))
for j in range(y.shape[0]):
    for k in range(z.shape[0]):
        for iy in range(4):
            for iz in range(4):
                spl = CubicSpline(x, coefftest[:, j, k, iy, iz])
                for i in range(x.shape[0]):
                    for ix in range(4):
                        dx = x[1] - x[0]
                        coefftest2[i, j, k, ix, iy, iz] = spl.coeff(i, ix)

In [9]:
def eval_func(xt, yt, zt):
    result = 0
    ix = int((xt - x[0])/(x[1] - x[0]))
    iy = int((yt - y[0])/(y[1] - y[0]))
    iz = int((zt - z[0])/(z[1] - z[0]))
    dx = (xt - x[0] - ix*(x[1] - x[0]))/(x[1] - x[0])
    dy = (yt - y[0] - iy*(y[1] - y[0]))/(y[1] - y[0])
    dz = (zt - z[0] - iz*(z[1] - z[0]))/(z[1] - z[0])
    print(ix, iy, iz, dx, dy, dz)
    for i in range(4):
        for j in range(4):
            for k in range(4):
                result += coefftest2[ix, iy, iz, i, j, k]*dx**i*dy**j*dz**k
    return result

In [20]:
%timeit trispl(0.8, 0.4, 2.4)

220 ns ± 5.37 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
